In [2]:
pip install transformers datasets

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 10.0 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 480 kB 24.8 MB/s eta 0:00:01
     |████████████████████████████████| 566 kB 26.2 MB/s eta 0:00:01
     |████████████████████████████████| 785 kB 20.9 MB/s eta 0:00:01
     |████████████████████████████████| 471 kB 73.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 53.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 77.3 MB/s eta 0:00:01
     |████████████████████████████████| 193 kB 57.8 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 9.1 MB/s  eta 0:00:01
     |████████████████████████████████| 132 kB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 179 kB 66.4 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 193 kB 30.2 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB

In [5]:
# =======================================================
# 🚀 终极下载方案：使用命令行 + 强制镜像
# =======================================================
# 这一步会将模型下载到当前目录下的 "wav2vec2_local" 文件夹中
# 即使没有梯子，在国内服务器通常也能跑满宽带
# =======================================================

!export HF_ENDPOINT=https://hf-mirror.com && huggingface-cli download \
    --resume-download facebook/wav2vec2-base-960h \
    --local-dir ./wav2vec2_local \
    --local-dir-use-symlinks False

/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/commands/download.py:141: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 11 files:   0%|                                 | 0/11 [00:00<?, ?it/s]Downloading 'model.safetensors' to 'wav2vec2_local/.cache/huggingface/download/xGOKKLRSlIhH692hSVvI1-gpoa8=.8aa76ab2243c81747a1f832954586bc566090c83a0ac167df6f31f0fa917d74a.incomplete'

feature_extractor_config.json: 158B [00:00, 571kB/s]A
Download complete. Moving file to wav2vec2_local/feature_extractor_config.json

config.json:

In [2]:
import os

print("=== 当前工作目录 ===")
print(os.getcwd())

print("\n=== 当前目录下的文件夹 ===")
print(os.listdir(".."))

# 检查你设置的路径是否存在
target_path = "../wav2vec2_local"  # 这是你报错信息里显示的路径
if os.path.exists(target_path):
    print(f"\n=== 目标文件夹 {target_path} 里的内容 ===")
    print(os.listdir(target_path))
else:
    print(f"\n❌ 错误：文件夹 '{target_path}' 根本不存在！")

=== 当前工作目录 ===
/root/VED

=== 当前目录下的文件夹 ===
['AudioWAV', '.ipynb_checkpoints', 'voiceEmotionDetective.ipynb', 'best_cnn.h5', 'best_crnn.h5', 'best_transformer.h5', 'voiceEmotionDetective-Copy1.ipynb', 'VED-CNN-Transfomer.ipynb', 'test.ipynb', 'best_cnn_transformer_ser.h5', 'best_conformer.h5', 'confusion_matrix_conformer.png', 'training_curves_conformer.png', 'class_f1_conformer.png', 'best_conformer_v2.h5', 'training_log_v2.csv', 'ConformerV1.ipynb', 'ConformerV3.1.ipynb', 'Output', 'DoubelStreamV4.ipynb', 'DoubelStreamV4.1.ipynb', 'ConformerV3.2.ipynb', 'ConformerV3.ipynb', 'ConformerV2.ipynb', 'ConformerV3.3.ipynb', 'wav2vec2', 'wav2vec2_local', 'v1.ipynb']

=== 目标文件夹 wav2vec2_local 里的内容 ===
['.cache', 'feature_extractor_config.json', 'config.json', '.gitattributes', 'preprocessor_config.json', 'README.md', 'tokenizer_config.json', 'vocab.json', 'special_tokens_map.json', 'tf_model.h5', 'model.safetensors', 'pytorch_model.bin']


In [1]:
# ============================================================================
# SER using Pre-trained Wav2Vec 2.0
# 策略: 使用 Wav2Vec 2.0 作为特征提取器 (Feature Extractor)
# 输入: Raw Audio (16kHz) -> Wav2Vec2 -> Dense Head -> Emotion
# ============================================================================

import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras import layers, models, callbacks, optimizers

# Hugging Face
from transformers import TFWav2Vec2Model, Wav2Vec2Processor

# 设置随机种子
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# =====================
# 1. 配置
# =====================
# 这里的模型可以换成 "microsoft/wavlm-base-plus" (效果更好，但需要兼容性检查)
MODEL_CHECKPOINT = "wav2vec2_local" 
AUDIO_DIR = Path("../AudioWAV")
BATCH_SIZE = 32 # 预训练模型显存占用大，建议从 8 或 16 开始
MAX_DURATION = 3.0 # 秒
SR = 16000 # 必须是 16k，这是预训练模型的规定

# 自动创建目录
timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
EXP_DIR = f"Output/wav2vec2_{timestamp}"
os.makedirs(f"{EXP_DIR}/models", exist_ok=True)

# =====================
# 2. 数据准备 (Raw Audio)
# =====================
def build_metadata(audio_dir):
    records = []
    for wav in audio_dir.glob("*.wav"):
        parts = wav.stem.split("_")
        if len(parts) == 4:
            records.append({"path": str(wav), "spk": parts[0], "emo": parts[2]})
    return pd.DataFrame(records)

meta = build_metadata(AUDIO_DIR)
print(f"Samples: {len(meta)}")

# 划分
le = LabelEncoder()
y = le.fit_transform(meta['emo'])
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
tr_idx, te_idx = next(gss.split(meta, y, groups=meta['spk']))
train_df = meta.iloc[tr_idx]
test_df = meta.iloc[te_idx]
y_tr, y_te = y[tr_idx], y[te_idx]

print(f"Train: {len(train_df)}, Test: {len(test_df)}")

# =====================
# 3. 数据生成器 (动态加载)
# =====================
# 预训练模型输入是原始波形，不能像 Log-Mel 一样全部读入内存（太占空间）
# 我们使用 Hugging Face 的 Processor 处理数据

# processor = Wav2Vec2Processor.from_pretrained(MODEL_CHECKPOINT)
processor = Wav2Vec2Processor.from_pretrained(MODEL_CHECKPOINT, local_files_only=True)
TARGET_LEN = int(SR * MAX_DURATION)

class AudioGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, y, batch_size=16, shuffle=True, augment=False):
        self.df = df
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.indexes = np.arange(len(df))
        if shuffle: np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, idx):
        batch_idx = self.indexes[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_paths = self.df.iloc[batch_idx]['path'].values
        batch_y = self.y[batch_idx]

        batch_x = []
        for p in batch_paths:
            # 读取音频
            speech, _ = librosa.load(p, sr=SR)
            
            # 简单数据增强 (可选)：随机裁剪或填充
            if self.augment and len(speech) > TARGET_LEN:
                start = np.random.randint(0, len(speech) - TARGET_LEN)
                speech = speech[start : start+TARGET_LEN]
            else:
                # 居中截断/填充
                if len(speech) > TARGET_LEN:
                    speech = speech[:TARGET_LEN]
                else:
                    pad_width = TARGET_LEN - len(speech)
                    speech = np.pad(speech, (0, pad_width))
            
            batch_x.append(speech)

        # 使用 Processor 进行标准化 (返回 tensor)
        # HuggingFace processor 负责归一化
        inputs = processor(batch_x, sampling_rate=SR, return_tensors="tf", padding=True)
        
        # 将标签转为 one-hot
        batch_y_oh = tf.keras.utils.to_categorical(batch_y, num_classes=len(le.classes_))
        
        return inputs.input_values, batch_y_oh

    def on_epoch_end(self):
        if self.shuffle: np.random.shuffle(self.indexes)

train_gen = AudioGenerator(train_df, y_tr, BATCH_SIZE, augment=True)
val_gen = AudioGenerator(test_df, y_te, BATCH_SIZE, shuffle=False, augment=False)

# =====================
# 4. 构建模型
# =====================
def build_wav2vec_model(num_classes):
    # 1. 加载预训练主体
    # from_pt=True 表示如果只有 PyTorch 权重，自动转换（第一次运行可能需要）
    # backbone = TFWav2Vec2Model.from_pretrained(MODEL_CHECKPOINT, from_pt=True)
    backbone = TFWav2Vec2Model.from_pretrained(MODEL_CHECKPOINT, from_pt=True, local_files_only=True)
    
    # 2. 冻结主体 (Feature Extraction 模式)
    # 如果想要 Fine-tuning，将这里设为 True，但需要极小的 LR
    backbone.trainable = False 
    
    # 3. 定义输入
    # Wav2Vec2 输入形状: (Batch, Time)
    input_audio = layers.Input(shape=(TARGET_LEN,), dtype=tf.float32, name="audio_input")
    
    # 4. 前向传播
    # backbone 返回一个对象，last_hidden_state 是 (Batch, Sequence, Hidden_Dim)
    # e.g., (Batch, 149, 768)
    outputs = backbone(input_audio)
    hidden_states = outputs.last_hidden_state
    
    # 5. 下游分类头 (Downstream Head)
    # 方法 A: Mean Pooling (对时间维度求平均)
    x = layers.GlobalAveragePooling1D()(hidden_states)
    
    # 方法 B: 也可以接 LSTM 或 Conformer，但通常 MLP 就够了
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    
    # Output
    prediction = layers.Dense(num_classes, activation="softmax", name="emotion")(x)
    
    model = models.Model(inputs=input_audio, outputs=prediction)
    return model

# =====================
# 5. 训练
# =====================
tf.keras.backend.clear_session()
model = build_wav2vec_model(len(le.classes_))
model.summary()

# 优化器：因为只训练分类头，LR 可以稍大 (1e-3 or 1e-4)
# 如果 backbone.trainable = True，LR 必须是 1e-5
optimizer = optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4)

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# 回调
class MacroF1Callback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # 为了速度，每 3 个 epoch 测一次，或者在验证集较大时抽样
        pass 
        # 注意：由于 Generator 每次生成一批，全量预测可能耗时，这里暂时依赖 val_accuracy

ckpt = callbacks.ModelCheckpoint(
    f"{EXP_DIR}/models/best_wav2vec.h5", 
    monitor="val_accuracy", 
    save_best_only=True, 
    save_weights_only=True, # HuggingFace 模型建议只存权重
    verbose=1
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30, # 预训练模型收敛通常很快
    callbacks=[ckpt, callbacks.EarlyStopping(patience=5, monitor="val_accuracy")],
    verbose=1
)

# =====================
# 6. 评估
# =====================
print("Evaluating...")
# 重新加载最佳权重
model.load_weights(f"{EXP_DIR}/models/best_wav2vec.h5")

# 预测
y_pred_indices = []
y_true_indices = []

for i in range(len(val_gen)):
    x_batch, y_batch = val_gen[i]
    preds = model.predict(x_batch, verbose=0)
    y_pred_indices.extend(np.argmax(preds, axis=1))
    y_true_indices.extend(np.argmax(y_batch, axis=1))

print(classification_report(y_true_indices, y_pred_indices, target_names=le.classes_, digits=4))

2025-12-04 09:14:13.628389: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 09:14:13.691056: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 09:14:14.706477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Samples: 7442
Train: 5890, Test: 1552


2025-12-04 09:14:18.845795: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31132 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:65:03.0, compute capability: 7.0

TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2025-12-04 09:14:20.187673: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2025-12-04 09:14:20.600394: I tensorflow/compiler/xla/service/service.cc:168] X

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 audio_input (InputLayer)    [(None, 48000)]           0         
                                                                 
 tf_wav2_vec2_model (TFWav2  TFWav2Vec2BaseModelOutp   94371712  
 Vec2Model)                  ut(last_hidden_state=(N             
                             one, 149, 768),                     
                              extract_features=(None             
                             , 149, 512),                        
                              hidden_states=None, at             
                             tentions=None)                      
                                                                 
 global_average_pooling1d (  (None, 768)               0         
 GlobalAveragePooling1D)                                         
                                                             

2025-12-04 09:14:49.092644: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


185/185 [==============================] - ETA: 0s - loss: 1.7484 - accuracy: 0.2329
Epoch 1: val_accuracy improved from -inf to 0.20232, saving model to Output/wav2vec2_20251204_091418/models/best_wav2vec.h5
185/185 [==============================] - 71s 283ms/step - loss: 1.7484 - accuracy: 0.2329 - val_loss: 1.9231 - val_accuracy: 0.2023
Epoch 2/30
185/185 [==============================] - ETA: 0s - loss: 1.6848 - accuracy: 0.2854
Epoch 2: val_accuracy did not improve from 0.20232
185/185 [==============================] - 47s 251ms/step - loss: 1.6848 - accuracy: 0.2854 - val_loss: 2.0174 - val_accuracy: 0.1907
Epoch 3/30
185/185 [==============================] - ETA: 0s - loss: 1.6602 - accuracy: 0.3024
Epoch 3: val_accuracy did not improve from 0.20232
185/185 [==============================] - 46s 250ms/step - loss: 1.6602 - accuracy: 0.3024 - val_loss: 1.9712 - val_accuracy: 0.2017
Epoch 4/30
185/185 [==============================] - ETA: 0s - loss: 1.6561 - accuracy: 0.3041

In [1]:
# ============================================================================
# SER using Pre-trained Wav2Vec 2.0 (Fine-Tuning Version)
# 策略: 解冻 Wav2Vec 2.0 进行微调 (Fine-Tuning)
# 针对环境: 离线模式 (Local Files) + 显存优化
# ============================================================================

import os
# 屏蔽 HF 下载警告
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras import layers, models, callbacks, optimizers

# Hugging Face
from transformers import TFWav2Vec2Model, Wav2Vec2Processor

# 设置随机种子
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 显存按需分配 (防止 OOM)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✓ GPU detected: {len(gpus)}")
    except RuntimeError as e:
        print(e)

# =====================
# 1. 配置
# =====================
# 请确保 MODEL_CHECKPOINT 指向你本地解压好的模型文件夹
MODEL_CHECKPOINT = "wav2vec2_local" 
AUDIO_DIR = Path("../AudioWAV")

# 微调时的 Batch Size 必须很小！
BATCH_SIZE = 32  # 如果显存还不够，改成 4
MAX_DURATION = 3.0 # 秒
SR = 16000 

# 自动创建目录
timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
EXP_DIR = f"Output/wav2vec2_finetune_{timestamp}"
os.makedirs(f"{EXP_DIR}/models", exist_ok=True)

# =====================
# 2. 数据准备 (Raw Audio)
# =====================
def build_metadata(audio_dir):
    records = []
    if not audio_dir.exists():
        print(f"Error: Directory {audio_dir} does not exist.")
        return pd.DataFrame()
        
    for wav in audio_dir.glob("*.wav"):
        parts = wav.stem.split("_")
        if len(parts) == 4:
            records.append({"path": str(wav), "spk": parts[0], "emo": parts[2]})
    return pd.DataFrame(records)

meta = build_metadata(AUDIO_DIR)
print(f"Samples: {len(meta)}")

if len(meta) == 0:
    raise ValueError("No audio files found. Check AUDIO_DIR path.")

# 划分
le = LabelEncoder()
y = le.fit_transform(meta['emo'])
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
tr_idx, te_idx = next(gss.split(meta, y, groups=meta['spk']))
train_df = meta.iloc[tr_idx]
test_df = meta.iloc[te_idx]
y_tr, y_te = y[tr_idx], y[te_idx]

print(f"Train: {len(train_df)}, Test: {len(test_df)}")

# =====================
# 3. 数据生成器
# =====================
# 加载本地 Processor
try:
    processor = Wav2Vec2Processor.from_pretrained(MODEL_CHECKPOINT, local_files_only=True)
    print("✓ Processor loaded from local path.")
except Exception as e:
    print(f"Error loading processor: {e}")
    # 如果本地加载失败，尝试在线加载（如果网络允许）或者报错停止
    raise e

TARGET_LEN = int(SR * MAX_DURATION)

class AudioGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, y, batch_size=8, shuffle=True, augment=False):
        self.df = df
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.indexes = np.arange(len(df))
        if shuffle: np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, idx):
        batch_idx = self.indexes[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_paths = self.df.iloc[batch_idx]['path'].values
        batch_y = self.y[batch_idx]

        batch_x = []
        for p in batch_paths:
            # 读取音频
            speech, _ = librosa.load(p, sr=SR)
            
            # 数据增强：简单的随机裁剪/平移
            if self.augment and len(speech) > TARGET_LEN:
                offset = np.random.randint(0, len(speech) - TARGET_LEN)
                speech = speech[offset : offset+TARGET_LEN]
            else:
                # 居中截断/填充
                if len(speech) > TARGET_LEN:
                    speech = speech[:TARGET_LEN]
                else:
                    pad_width = TARGET_LEN - len(speech)
                    speech = np.pad(speech, (0, pad_width))
            
            batch_x.append(speech)

        # 使用 Processor 进行归一化
        # return_tensors="tf" 返回 TF Tensor
        inputs = processor(batch_x, sampling_rate=SR, return_tensors="tf", padding=True)
        
        # 标签 one-hot
        batch_y_oh = tf.keras.utils.to_categorical(batch_y, num_classes=len(le.classes_))
        
        return inputs.input_values, batch_y_oh

    def on_epoch_end(self):
        if self.shuffle: np.random.shuffle(self.indexes)

train_gen = AudioGenerator(train_df, y_tr, BATCH_SIZE, augment=True)
val_gen = AudioGenerator(test_df, y_te, BATCH_SIZE, shuffle=False, augment=False)

# =====================
# 4. 构建可微调模型
# =====================
def build_finetune_model(num_classes):
    # 1. 加载预训练主体 (本地)
    try:
        # from_pt=True 如果你的本地文件是 pytorch_model.bin
        # 如果是 tf_model.h5，则不需要 from_pt=True
        # 这里假设通用的 PyTorch 权重
        backbone = TFWav2Vec2Model.from_pretrained(MODEL_CHECKPOINT, from_pt=True, local_files_only=True)
        print("✓ Model loaded from local path.")
    except OSError:
        # 尝试不加 from_pt (如果是 TF 权重)
        backbone = TFWav2Vec2Model.from_pretrained(MODEL_CHECKPOINT, local_files_only=True)
    
    # 2. === 关键 === 解冻模型进行微调
    backbone.trainable = True 
    
    # 3. 定义输入
    input_audio = layers.Input(shape=(TARGET_LEN,), dtype=tf.float32, name="audio_input")
    
    # 4. 前向传播
    outputs = backbone(input_audio)
    # last_hidden_state: (Batch, Time, 768)
    x = outputs.last_hidden_state 
    
    # 5. 改进的分类头：Attention Pooling
    # 相比 GlobalAveragePooling，Attention 能自动关注情感强烈的片段
    # 投影到较低维度计算 score
    att_score = layers.Dense(1, activation="tanh")(x) # (B, T, 1)
    att_score = layers.Flatten()(att_score)           # (B, T)
    att_weight = layers.Activation("softmax")(att_score) # (B, T)
    
    # 加权求和
    # Expand dims for broadcasting: (B, T, 1) * (B, T, 768)
    att_weight = layers.Reshape((-1, 1))(att_weight)
    x = layers.Multiply()([x, att_weight])
    x = tf.reduce_sum(x, axis=1) # (B, 768)
    
    # MLP
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.2)(x) # 微调时 Dropout 不宜过大
    x = layers.Dense(128, activation="relu")(x)
    
    # Output
    prediction = layers.Dense(num_classes, activation="softmax", name="emotion")(x)
    
    model = models.Model(inputs=input_audio, outputs=prediction)
    return model

tf.keras.backend.clear_session()
model = build_finetune_model(len(le.classes_))
model.summary()

# =====================
# 5. 编译与训练
# =====================
# ⚠️ 关键设置：学习率必须非常小 (1e-5)
# 如果太大，会瞬间破坏预训练权重，导致 Loss 不降反升
optimizer = optimizers.Adam(learning_rate=1e-5)

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

ckpt = callbacks.ModelCheckpoint(
    f"{EXP_DIR}/models/best_wav2vec.h5", 
    monitor="val_accuracy", 
    save_best_only=True, 
    save_weights_only=True, # 对于 HuggingFace 模型，只保存权重最安全
    verbose=1
)

# 学习率衰减
lr_schedule = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

print("\nStarting Fine-Tuning (This will be slower)...")
# 微调通常只需要很少的 Epoch (5-10) 就能收敛
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15, 
    callbacks=[ckpt, lr_schedule, early_stop],
    verbose=1
)

# =====================
# 6. 评估
# =====================
print("Evaluating...")
# 重新加载最佳权重
model.load_weights(f"{EXP_DIR}/models/best_wav2vec.h5")

y_pred_indices = []
y_true_indices = []

# 批量预测
for i in range(len(val_gen)):
    x_batch, y_batch = val_gen[i]
    preds = model.predict(x_batch, verbose=0)
    y_pred_indices.extend(np.argmax(preds, axis=1))
    y_true_indices.extend(np.argmax(y_batch, axis=1))

print(classification_report(y_true_indices, y_pred_indices, target_names=le.classes_, digits=4))

2025-12-04 09:33:55.210264: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 09:33:55.277994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 09:33:56.224365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✓ GPU detected: 1
Samples: 7442
Train: 5890, Test: 1552
✓ Processor loaded from local path.


2025-12-04 09:34:00.212756: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31132 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:65:03.0, compute capability: 7.0

TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2025-12-04 09:34:01.113746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2025-12-04 09:34:01.529392: I tensorflow/compiler/xla/service/service.cc:168] X

✓ Model loaded from local path.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 audio_input (InputLayer)    [(None, 48000)]              0         []                            
                                                                                                  
 tf_wav2_vec2_model (TFWav2  TFWav2Vec2BaseModelOutput(   9437171   ['audio_input[0][0]']         
 Vec2Model)                  last_hidden_state=(None, 1   2                                       
                             49, 768),                                                            
                              extract_features=(None, 1                                           
                             49, 512),                                                            
                              hidden_states=None, atten       

2025-12-04 09:34:43.726345: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


185/185 [==============================] - ETA: 0s - loss: 1.6784 - accuracy: 0.2900
Epoch 1: val_accuracy improved from -inf to 0.29188, saving model to Output/wav2vec2_finetune_20251204_093400/models/best_wav2vec.h5
185/185 [==============================] - 173s 644ms/step - loss: 1.6784 - accuracy: 0.2900 - val_loss: 1.6633 - val_accuracy: 0.2919 - lr: 1.0000e-05
Epoch 2/15
185/185 [==============================] - ETA: 0s - loss: 1.4547 - accuracy: 0.4190
Epoch 2: val_accuracy improved from 0.29188 to 0.36534, saving model to Output/wav2vec2_finetune_20251204_093400/models/best_wav2vec.h5
185/185 [==============================] - 113s 611ms/step - loss: 1.4547 - accuracy: 0.4190 - val_loss: 1.6049 - val_accuracy: 0.3653 - lr: 1.0000e-05
Epoch 3/15
185/185 [==============================] - ETA: 0s - loss: 1.2893 - accuracy: 0.5163
Epoch 3: val_accuracy improved from 0.36534 to 0.37693, saving model to Output/wav2vec2_finetune_20251204_093400/models/best_wav2vec.h5
185/185 [=====

In [1]:
# ============================================================================
# Wav2Vec 2.0 Fine-Tuning (Advanced)
# 架构: Raw Audio -> Wav2Vec2 (Fine-tune) -> BiLSTM -> Attention -> Emotion
# 改进: 更强的分类头 + 更长的训练 + 分层学习率
# ============================================================================

import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models, callbacks, optimizers

from transformers import TFWav2Vec2Model, Wav2Vec2Processor

# 显存配置
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e: print(e)

# 1. 配置
MODEL_CHECKPOINT = "wav2vec2_local" 
AUDIO_DIR = Path("../AudioWAV")
BATCH_SIZE = 64  # 保持 8 以防 OOM
MAX_DURATION = 3.0 
SR = 16000

timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
EXP_DIR = f"Output/wav2vec2_lstm_{timestamp}"
os.makedirs(f"{EXP_DIR}/models", exist_ok=True)

# 2. 数据准备
def build_metadata(audio_dir):
    records = []
    for wav in audio_dir.glob("*.wav"):
        p = wav.stem.split("_")
        if len(p) == 4: records.append({"path": str(wav), "spk": p[0], "emo": p[2]})
    return pd.DataFrame(records)

meta = build_metadata(AUDIO_DIR)
le = LabelEncoder()
y = le.fit_transform(meta['emo'])
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr_idx, te_idx = next(gss.split(meta, y, groups=meta['spk']))
train_df, test_df = meta.iloc[tr_idx], meta.iloc[te_idx]
y_tr, y_te = y[tr_idx], y[te_idx]

print(f"Train: {len(train_df)}, Test: {len(test_df)}")

# 3. Generator
processor = Wav2Vec2Processor.from_pretrained(MODEL_CHECKPOINT, local_files_only=True)
TARGET_LEN = int(SR * MAX_DURATION)

class AudioGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, y, batch_size=8, shuffle=True, augment=False):
        self.df, self.y = df, y
        self.batch_size, self.shuffle = batch_size, shuffle
        self.augment = augment
        self.indexes = np.arange(len(df))
        if shuffle: np.random.shuffle(self.indexes)

    def __len__(self): return int(np.ceil(len(self.df)/self.batch_size))

    def __getitem__(self, idx):
        inds = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x, batch_y = [], self.y[inds]
        
        for p in self.df.iloc[inds]['path'].values:
            speech, _ = librosa.load(p, sr=SR)
            # Augment
            if self.augment and len(speech) > TARGET_LEN:
                off = np.random.randint(0, len(speech)-TARGET_LEN)
                speech = speech[off:off+TARGET_LEN]
            else:
                if len(speech) > TARGET_LEN: speech = speech[:TARGET_LEN]
                else: speech = np.pad(speech, (0, TARGET_LEN-len(speech)))
            batch_x.append(speech)
            
        inputs = processor(batch_x, sampling_rate=SR, return_tensors="tf", padding=True)
        return inputs.input_values, tf.keras.utils.to_categorical(batch_y, num_classes=len(le.classes_))
    
    def on_epoch_end(self):
        if self.shuffle: np.random.shuffle(self.indexes)

train_gen = AudioGenerator(train_df, y_tr, BATCH_SIZE, augment=True)
val_gen = AudioGenerator(test_df, y_te, BATCH_SIZE, shuffle=False)

# 4. 模型构建 (Wav2Vec + BiLSTM)
def build_model(num_classes):
    # Backbone
    backbone = TFWav2Vec2Model.from_pretrained(MODEL_CHECKPOINT, from_pt=True, local_files_only=True)
    backbone.trainable = True # 微调
    
    inp = layers.Input(shape=(TARGET_LEN,), dtype=tf.float32)
    x = backbone(inp).last_hidden_state # (B, T, 768)
    
    # --- 增强的分类头 ---
    # 1. 降维 (节省显存)
    x = layers.Dense(256)(x) 
    x = layers.Dropout(0.1)(x)
    
    # 2. Bi-LSTM (捕捉时序依赖，解决 SAD/DIS)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)
    
    # 3. Classifier
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    
    return models.Model(inputs=inp, outputs=out)

tf.keras.backend.clear_session()
model = build_model(len(le.classes_))
model.summary()

# 5. 训练
# 学习率设置：这里我们用一个稍大的 LR (5e-5)，配合 Warmup
optimizer = optimizers.AdamW(learning_rate=5e-5, weight_decay=1e-4)

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Callbacks
ckpt = callbacks.ModelCheckpoint(f"{EXP_DIR}/models/best.h5", monitor="val_accuracy", save_best_only=True, save_weights_only=True, verbose=1)
# 增加耐心值
early = callbacks.EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True)
# LR 衰减
reduce = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-7)

print("\nStarting Advanced Fine-Tuning (60 Epochs)...")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=60, # 增加训练轮数
    callbacks=[ckpt, early, reduce],
    verbose=1
)

# 6. 评估
print("Evaluating...")
model.load_weights(f"{EXP_DIR}/models/best.h5")
yp, yt = [], []
for i in range(len(val_gen)):
    x, y = val_gen[i]
    p = model.predict(x, verbose=0)
    yp.extend(np.argmax(p, axis=1))
    yt.extend(np.argmax(y, axis=1))

print(classification_report(yt, yp, target_names=le.classes_, digits=4))

2025-12-04 10:14:28.303361: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 10:14:28.370076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 10:14:29.309254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-12-04 10:14:33.240851: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31132 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:65:03.0, compute capabi

Train: 5890, Test: 1552


/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2025-12-04 10:14:34.115625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2025-12-04 10:14:34.518323: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55eacbd41650 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-04 10:14:34.518357: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2025-12-04 10:14:34.872472: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48000)]           0         
                                                                 
 tf_wav2_vec2_model (TFWav2  TFWav2Vec2BaseModelOutp   94371712  
 Vec2Model)                  ut(last_hidden_state=(N             
                             one, 149, 768),                     
                              extract_features=(None             
                             , 149, 512),                        
                              hidden_states=None, at             
                             tentions=None)                      
                                                                 
 dense (Dense)               (None, 149, 256)          196864    
                                                                 
 dropout_50 (Dropout)        (None, 149, 256)          0     

2025-12-04 10:15:24.764779: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


93/93 [==============================] - ETA: 0s - loss: 1.5605 - accuracy: 0.3659
Epoch 1: val_accuracy improved from -inf to 0.29832, saving model to Output/wav2vec2_lstm_20251204_101433/models/best.h5
93/93 [==============================] - 167s 1s/step - loss: 1.5605 - accuracy: 0.3659 - val_loss: 1.6984 - val_accuracy: 0.2983 - lr: 5.0000e-05
Epoch 2/60
93/93 [==============================] - ETA: 0s - loss: 1.0449 - accuracy: 0.6311
Epoch 3: val_accuracy improved from 0.35760 to 0.39111, saving model to Output/wav2vec2_lstm_20251204_101433/models/best.h5
93/93 [==============================] - 101s 1s/step - loss: 1.0449 - accuracy: 0.6311 - val_loss: 1.6073 - val_accuracy: 0.3911 - lr: 5.0000e-05
Epoch 4/60
93/93 [==============================] - ETA: 0s - loss: 0.9778 - accuracy: 0.6637
Epoch 4: val_accuracy did not improve from 0.39111
93/93 [==============================] - 99s 1s/step - loss: 0.9778 - accuracy: 0.6637 - val_loss: 1.5968 - val_accuracy: 0.3731 - lr: 5.00